In [1]:
%pip install numpy
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Очистка и подготовка данных парсинга исторических данных о погоде к дальнейшей работе
После парсинга и объединения данных у нас есть общий файл для всех городов России с информацией о погодных условиях. 
Сейчас нужно проверить качество - изучить типы данных, пропуски, дубликаты, обрезать данные, чтобы они шли равномерно для всех городов.


### 1. Загрузка библиотек и изучение данных

In [31]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('D:\\Datas\\meteo\\final\\union_all_weather.csv')

In [4]:
def preprocessing(dataframe):
    display(dataframe.head(5))
    print('-'*50)
    dataframe.info()
    print('-'*50)
    display(dataframe.describe())
    print('-'*50)
    print(f"Явных дубликатов: {dataframe.duplicated().sum()}")
    print('-'*50)
    print('Пропусков в колонках:')
    print(dataframe.isna().sum())
    print('-'*50)
    print('Размер таблицы')
    print(dataframe.shape)

In [5]:
preprocessing(df)

,Unnamed: 0,city,temperature_2m,relative_humidity_2m,rain,snowfall,snow_depth,is_day,precipitation,wind_direction_100m,wind_speed_100m,date
0,0,Алзамай,-19.518501,82.294174,0.0,0.0,0.6,False,0.0,165.96373,1.484318,1975-01-01 00:00:00
1,1,Алзамай,-16.518501,76.592300,0.0,0.0,0.6,True,0.0,203.96250,3.545589,1975-01-01 06:00:00
2,2,Алзамай,-19.418500,82.308586,0.0,0.0,0.6,False,0.0,212.61923,10.685391,1975-01-01 12:00:00
3,3,Алзамай,-22.318500,79.705460,0.0,0.0,0.6,False,0.0,208.44283,9.826088,1975-01-01 18:00:00
4,4,Алзамай,-23.418500,78.803050,0.0,0.0,0.6,False,0.0,226.27295,11.457958,1975-01-02 00:00:00


--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81967860 entries, 0 to 81967859
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Unnamed: 0            int64  
 1   city                  object 
 2   temperature_2m        float64
 3   relative_humidity_2m  float64
 4   rain                  float64
 5   snowfall              float64
 6   snow_depth            float64
 7   is_day                bool   
 8   precipitation         float64
 9   wind_direction_100m   float64
 10  wind_speed_100m       float64
 11  date                  object 
dtypes: bool(1), float64(8), int64(1), object(2)
memory usage: 6.8+ GB
--------------------------------------------------


,Unnamed: 0,temperature_2m,relative_humidity_2m,rain,snowfall,snow_depth,precipitation,wind_direction_100m,wind_speed_100m
count,8.196786e+07,8.196715e+07,8.196715e+07,8.196715e+07,8.196715e+07,8.120879e+07,8.196715e+07,8.196715e+07,8.196715e+07
mean,4.156722e+07,4.642497e+00,7.554121e+01,2.647520e-01,7.180459e-02,1.371049e-01,3.575182e-01,1.955356e+02,2.095345e+01
std,2.437845e+07,1.284415e+01,1.619038e+01,1.088770e+00,3.299326e-01,2.291683e-01,1.180840e+00,9.775226e+01,9.925938e+00
min,0.000000e+00,-5.990000e+01,4.793310e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.052047e+07,-3.520000e+00,6.592496e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.165650e+02,1.375558e+01
50%,4.108608e+07,5.087000e+00,7.905740e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.087675e+02,2.046941e+01
75%,6.208941e+07,1.471500e+01,8.800230e+01,0.000000e+00,0.000000e+00,2.200000e-01,1.000000e-01,2.742053e+02,2.723894e+01
max,8.426780e+07,4.294000e+01,1.000000e+02,9.220001e+01,2.261000e+01,2.960000e+00,9.220001e+01,3.600000e+02,1.199843e+02


--------------------------------------------------
Явных дубликатов: 0
--------------------------------------------------
Пропусков в колонках:
Unnamed: 0                   0
city                         0
temperature_2m             714
relative_humidity_2m       714
rain                       714
snowfall                   714
snow_depth              759074
is_day                       0
precipitation              714
wind_direction_100m        714
wind_speed_100m            714
date                         0
dtype: int64
--------------------------------------------------
Размер таблицы
(81967860, 12)


При изучении данных обнаружены:
- лишняя колонка с индексами;
- типы данных некорректны;
- есть 714 пропусков данных в нескольких колонках - это знак, что нужно проверить эти пропуски и вероятно удалить;
- явных дубликатов нет, но можно проверить более настраиваемые дубликаты.

In [6]:
df_original = df.copy()

## Очистка данных

### Удаляем лишнюю колонку

In [7]:
df = df.drop(columns=["Unnamed: 0"], errors='ignore')

### Изменение типа данных

In [8]:
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['city', 'date'])
print(df.dtypes)

city                            object
temperature_2m                 float64
relative_humidity_2m           float64
rain                           float64
snowfall                       float64
snow_depth                     float64
is_day                            bool
precipitation                  float64
wind_direction_100m            float64
wind_speed_100m                float64
date                    datetime64[ns]
dtype: object


### Разбираемся с неочевидными дубликатами

In [9]:
duplicates = df.duplicated(subset=['city', 'date'], keep=False)
print(f"\nНайдено {duplicates.sum()} дубликатов по городу и дате")


Найдено 689375 дубликатов по городу и дате


In [10]:
df = df.drop_duplicates(subset=['city', 'date'], keep='first')

### Обрезаем даты

In [11]:
def filter_data_by_city_dates(df):
    min_dates_per_city = df.groupby('city')['date'].min()
    cutoff_min = min_dates_per_city.max()
    df = df[df['date'] >= cutoff_min]
    
    max_dates_per_city = df.groupby('city')['date'].max()
    cutoff_max = max_dates_per_city.max()
    df = df[df['date'] <= cutoff_max]
    
    actual_min_date = df['date'].min()
    actual_max_date = df['date'].max()
    
    print(f"\nФактическая минимальная дата в результате: {actual_min_date}")
    print(f"Фактическая максимальная дата в результате: {actual_max_date}")
    
    return df, actual_min_date, actual_max_date

In [12]:
df, min_date, max_date = filter_data_by_city_dates(df)


Фактическая минимальная дата в результате: 1985-05-30 00:00:00
Фактическая максимальная дата в результате: 2025-06-05 18:00:00


### Обработка пропусков

Т.к. много пропусков в колонке уровень снега - это значит, что в это время снега не было и соответственно можно заполнить пропуски "0".

In [13]:
df['snow_depth'] = df['snow_depth'].fillna(0)

Проверим характер расположения других пропусков:

In [14]:
max_nans_per_column = {}

for col in df.columns:
    consecutive_nans = (
        df[col].isnull()  # Находим пропуски (True/False)
        .astype(int)              # Преобразуем в 1 (пропуск) и 0 (не пропуск)
        .groupby(                  # Группируем по группам подряд идущих пропусков
            df[col].notnull().astype(int).cumsum()  # Нумеруем группы не-NaN значений
        )
        .sum()                    # Суммируем количество пропусков в каждой группе
    )
    max_consecutive = consecutive_nans.max()  # Находим максимум
    max_nans_per_column[col] = max_consecutive  # Сохраняем результат

# Сортируем столбцы по убыванию максимального числа пропусков
sorted_results = sorted(
    max_nans_per_column.items(),
    key=lambda x: x[1],
    reverse=True
)

# Выводим результаты
for col, max_nans in sorted_results:
    print(f"{col}: Максимальное количество подряд идущих пропусков — {max_nans}")

temperature_2m: Максимальное количество подряд идущих пропусков — 11
relative_humidity_2m: Максимальное количество подряд идущих пропусков — 11
rain: Максимальное количество подряд идущих пропусков — 11
snowfall: Максимальное количество подряд идущих пропусков — 11
precipitation: Максимальное количество подряд идущих пропусков — 11
wind_direction_100m: Максимальное количество подряд идущих пропусков — 11
wind_speed_100m: Максимальное количество подряд идущих пропусков — 11
city: Максимальное количество подряд идущих пропусков — 0
snow_depth: Максимальное количество подряд идущих пропусков — 0
is_day: Максимальное количество подряд идущих пропусков — 0
date: Максимальное количество подряд идущих пропусков — 0


Проверим в одинаковых ли строках находятся пропуски в кол-ве 714

In [15]:
columns_to_check = ['temperature_2m', 'relative_humidity_2m', 'rain', 'snowfall', 'wind_speed_100m', 'wind_direction_100m', 'precipitation']
masks = [df[col].isna() for col in columns_to_check]
all_equal = all(mask.equals(masks[0]) for mask in masks)

print("Все маски идентичны:", all_equal)

Все маски идентичны: True


Строки одинаковы, растояние между ними не более 11 пустых строк. Исходя из этого заполним пропуски с помощью временных рядов и предсказания с помощью RandomForestRegressor

In [16]:
df = df.set_index('date', drop=False)
# 3. Заполнение пропусков в признаках
features = ['relative_humidity_2m', 'rain', 'snowfall', 'wind_speed_100m', 'wind_direction_100m', 'precipitation']
for col in features:
    df[col] = df.groupby(['city', df.index.hour])[col].transform(
        lambda x: x.fillna(x.median())
    )

# 4. Заполнение температуры с учетом времени
def fill_weather_data(group):
    # Убедимся, что группа имеет DatetimeIndex
    group = group.set_index('date')  
    
    # Интерполяция по времени
    group['temperature_2m'] = group['temperature_2m'].interpolate(method='time')
    
    # ML-предсказание (если в features нет пропусков)
    if group['temperature_2m'].isnull().any() and not group[features].isnull().any().any():
        known = group[group['temperature_2m'].notna()]
        unknown = group[group['temperature_2m'].isna()]
        
        if len(known) > 10:
            model = RandomForestRegressor(n_estimators=50, random_state=42)
            model.fit(known[features], known['temperature_2m'])
            group.loc[unknown.index, 'temperature_2m'] = model.predict(unknown[features])
    
    # Заполнение медианой по времени суток
    group['temperature_2m'] = group.groupby(group.index.hour)['temperature_2m'].transform(
        lambda x: x.fillna(x.median()))
    
    return group.reset_index()  # возвращаем 'date' как колонку

# Применяем обработку для каждого города
df = df.groupby('city', group_keys=False).apply(fill_weather_data)

# 5. Проверка результатов
print("\nКоличество пропусков после обработки:")
print(df.isnull().sum())



Количество пропусков после обработки:
date                    0
city                    0
temperature_2m          0
relative_humidity_2m    0
rain                    0
snowfall                0
snow_depth              0
is_day                  0
precipitation           0
wind_direction_100m     0
wind_speed_100m         0
dtype: int64


Все пропуски заполнены

## Проверка полноты данных

In [17]:
city_df = pd.read_csv('city_data_good.csv')

In [18]:
# Находим различия в обоих направлениях
cities_in_cities = set(city_df['name'])
cities_in_weather = set(df['city'])

# 1. Города, которые есть в таблице городов, но нет в погоде
missing_in_weather = sorted(cities_in_cities - cities_in_weather)

# 2. Города, которые есть в погоде, но нет в таблице городов
extra_in_weather = sorted(cities_in_weather - cities_in_cities)

# Выводим результаты
if missing_in_weather:
    print(f"\nГорода из основной таблицы, отсутствующие в таблице погоды ({len(missing_in_weather)} шт.):")
    print(*missing_in_weather, sep=", ")
else:
    print("Все города из основной таблицы есть в таблице погоды.")

if extra_in_weather:
    print(f"\nГорода из таблицы погоды, отсутствующие в основной таблице ({len(extra_in_weather)} шт.):")
    print(*extra_in_weather, sep=", ")
else:
    print("\nВ таблице погоды нет лишних городов (все есть в основной таблице).")

# Дополнительно: сводка по пересечению
common_cities = sorted(cities_in_cities & cities_in_weather)
print(f"\nГорода, которые есть в обеих таблицах ({len(common_cities)} шт.):")
print(*common_cities, sep=", ")


Города из основной таблицы, отсутствующие в таблице погоды (66 шт.):
Балашиха, Благовещенск, Верхний Уфалей, Всеволожск, Горячий Ключ, Дегтярск, Ессентуки, Железногорск, Жердевка, Западная Двина, Калуга, Карачаевск, Карпинск, Кемерово, Кизел, Кимовск, Киржач, Кировск, Кореновск, Коркино, Красноармейск, Краснознаменск, Краснослободск, Лесной, Мегион, Мезень, Миллерово, Надым, Нефтеюганск, Нижний Ломов, Новосибирск, Новотроицк, Новый Уренгой, Ногинск, Обнинск, Октябрьский, Орёл, Приволжск, Пугачёв, Пудож, Реутов, Россошь, Рузаевка, Ряжск, Свирск, Североморск, Сергач, Сертолово, Советск, Советская Гавань, Среднеуральск, Стрежевой, Тимашёвск, Туапсе, Уварово, Углегорск, Урюпинск, Ухта, Фокино, Чайковский, Чистополь, Шахунья, Шимановск, Южно-Сухокумск, Юрьевец, Ясный

Города из таблицы погоды, отсутствующие в основной таблице (7 шт.):
Верхний_Уфалей, Горячий_Ключ, Есентуки, Мезень (город), Октябрьский (город), Пугачёв (город), Советская_Гавань

Города, которые есть в обеих таблицах (1039 ш

In [19]:
city_name_corrections = {
    'Верхний_Уфалей': 'Верхний Уфалей',
    'Горячий_Ключ': 'Горячий Ключ',
    'Есентуки': 'Ессентуки',
    'Мезень (город)': 'Мезень',
    'Октябрьский (город)': 'Октябрьский',
    'Пугачёв (город)': 'Пугачёв',
    'Советская_Гавань': 'Советская Гавань'
    # Добавьте другие замены по необходимости
}

# Применяем замену названий в таблице погоды
df['city'] = df['city'].replace(city_name_corrections)

cities_in_cities = set(city_df['name'])
cities_in_weather = set(df['city'])

# 1. Города, которые есть в таблице городов, но нет в погоде
missing_in_weather = sorted(cities_in_cities - cities_in_weather)

# 2. Города, которые есть в погоде, но нет в таблице городов
extra_in_weather = sorted(cities_in_weather - cities_in_cities)

# Выводим результаты
if missing_in_weather:
    print(f"\nГорода из основной таблицы, отсутствующие в таблице погоды ({len(missing_in_weather)} шт.):")
    print(*missing_in_weather, sep=", ")
else:
    print("Все города из основной таблицы есть в таблице погоды.")

if extra_in_weather:
    print(f"\nГорода из таблицы погоды, отсутствующие в основной таблице ({len(extra_in_weather)} шт.):")
    print(*extra_in_weather, sep=", ")
else:
    print("\nВ таблице погоды нет лишних городов (все есть в основной таблице).")

# Дополнительно: сводка по пересечению
common_cities = sorted(cities_in_cities & cities_in_weather)
print(f"\nГорода, которые есть в обеих таблицах ({len(common_cities)} шт.):")
print(*common_cities, sep=", ")


Города из основной таблицы, отсутствующие в таблице погоды (59 шт.):
Балашиха, Благовещенск, Всеволожск, Дегтярск, Железногорск, Жердевка, Западная Двина, Калуга, Карачаевск, Карпинск, Кемерово, Кизел, Кимовск, Киржач, Кировск, Кореновск, Коркино, Красноармейск, Краснознаменск, Краснослободск, Лесной, Мегион, Миллерово, Надым, Нефтеюганск, Нижний Ломов, Новосибирск, Новотроицк, Новый Уренгой, Ногинск, Обнинск, Орёл, Приволжск, Пудож, Реутов, Россошь, Рузаевка, Ряжск, Свирск, Североморск, Сергач, Сертолово, Советск, Среднеуральск, Стрежевой, Тимашёвск, Туапсе, Уварово, Углегорск, Урюпинск, Ухта, Фокино, Чайковский, Чистополь, Шахунья, Шимановск, Южно-Сухокумск, Юрьевец, Ясный

В таблице погоды нет лишних городов (все есть в основной таблице).

Города, которые есть в обеих таблицах (1046 шт.):
Абаза, Абакан, Абдулино, Абинск, Агидель, Агрыз, Адыгейск, Азнакаево, Азов, Ак-Довурак, Аксай, Алагир, Алапаевск, Алатырь, Алдан, Алейск, Александров, Александровск, Александровск-Сахалинский, Але

In [20]:
preprocessing(df)

,date,city,temperature_2m,relative_humidity_2m,rain,snowfall,snow_depth,is_day,precipitation,wind_direction_100m,wind_speed_100m
0,1985-05-30 00:00:00,Абаза,6.22,87.941080,0.0,0.0,0.0,True,0.0,190.30480,8.049845
1,1985-05-30 06:00:00,Абаза,18.81,34.609940,0.0,0.0,0.0,True,0.0,143.47105,12.096214
2,1985-05-30 12:00:00,Абаза,17.31,42.712345,0.1,0.0,0.0,True,0.1,267.22986,22.346113
3,1985-05-30 18:00:00,Абаза,9.57,87.940310,0.8,0.0,0.0,False,0.8,219.05313,24.569836
4,1985-05-31 00:00:00,Абаза,8.67,95.041270,0.0,0.0,0.0,True,0.0,160.20103,9.565437


--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Index: 61638479 entries, 0 to 58463
Data columns (total 11 columns):
 #   Column                Dtype         
---  ------                -----         
 0   date                  datetime64[ns]
 1   city                  object        
 2   temperature_2m        float64       
 3   relative_humidity_2m  float64       
 4   rain                  float64       
 5   snowfall              float64       
 6   snow_depth            float64       
 7   is_day                bool          
 8   precipitation         float64       
 9   wind_direction_100m   float64       
 10  wind_speed_100m       float64       
dtypes: bool(1), datetime64[ns](1), float64(8), object(1)
memory usage: 5.1+ GB
--------------------------------------------------


,date,temperature_2m,relative_humidity_2m,rain,snowfall,snow_depth,precipitation,wind_direction_100m,wind_speed_100m
count,61638479,6.163848e+07,6.163848e+07,6.163848e+07,6.163848e+07,6.163848e+07,6.163848e+07,6.163848e+07,6.163848e+07
mean,2005-04-25 00:37:57.103030912,4.949646e+00,7.526850e+01,2.635818e-01,6.991531e-02,1.300700e-01,3.528795e-01,1.957515e+02,2.091326e+01
min,1985-05-30 00:00:00,-5.871950e+01,4.793310e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1995-05-12 12:00:00,-3.141500e+00,6.533850e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.170126e+02,1.370839e+01
50%,2005-04-24 00:00:00,5.382500e+00,7.894501e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.090000e+02,2.042187e+01
75%,2015-04-07 00:00:00,1.491950e+01,8.800000e+01,0.000000e+00,0.000000e+00,2.000000e-01,1.000000e-01,2.742363e+02,2.721276e+01
max,2025-06-05 18:00:00,4.294000e+01,1.000000e+02,9.220001e+01,2.198000e+01,2.960000e+00,9.220001e+01,3.600000e+02,1.174974e+02
std,NaN,1.276031e+01,1.641413e+01,1.095737e+00,3.266634e-01,2.227320e-01,1.185963e+00,9.766218e+01,9.918695e+00


--------------------------------------------------
Явных дубликатов: 0
--------------------------------------------------
Пропусков в колонках:
date                    0
city                    0
temperature_2m          0
relative_humidity_2m    0
rain                    0
snowfall                0
snow_depth              0
is_day                  0
precipitation           0
wind_direction_100m     0
wind_speed_100m         0
dtype: int64
--------------------------------------------------
Размер таблицы
(61638479, 11)


In [21]:
# Находим города, которые есть в cities_df, но отсутствуют в weather_df
missing_cities = city_df[~city_df['name'].isin(df['city'])]

# Проверяем, есть ли среди них города из Иркутской области
irkutsk_missing = missing_cities[missing_cities['region'] == 'Иркутская область']

if not irkutsk_missing.empty:
    print("Города из Иркутской области, отсутствующие в таблице погоды:")
    print(irkutsk_missing[['name', 'region']])
else:
    print("В таблице погоды есть все города Иркутской области из основной таблицы.")

Города из Иркутской области, отсутствующие в таблице погоды:
       name             region
845  Свирск  Иркутская область


### Создание среза по Иркутской области

In [22]:
city_ids = dict(zip(city_df['name'], city_df['id']))

# Заменяем названия городов на ID
df['city_id'] = df['city'].map(city_ids)
df.drop('city', axis=1, inplace=True)  # удаляем столбец с названиями городов

In [23]:
# Сохраняем обработанные данные
df.to_csv('processed_weather_data.csv', index=False)
print("\nОбработка данных завершена. Результат сохранен в processed_weather_data.csv")


Обработка данных завершена. Результат сохранен в processed_weather_data.csv


In [24]:
target_region = 'Иркутская область'

# Получаем список городов из нужного региона
cities_in_region = city_df[city_df['region'] == target_region]['id'].tolist()

# Делаем срез таблицы погоды только для этих городов
weather_slice = df[df['city_id'].isin(cities_in_region)]

display(weather_slice.head())

,date,temperature_2m,relative_humidity_2m,rain,snowfall,snow_depth,is_day,precipitation,wind_direction_100m,wind_speed_100m,city_id
0,1985-05-30 00:00:00,10.7315,68.325270,0.0,0.0,0.0,True,0.0,259.56256,13.910169,22
1,1985-05-30 06:00:00,18.4315,39.305890,0.0,0.0,0.0,True,0.0,260.53775,4.379589,22
2,1985-05-30 12:00:00,16.8315,55.358337,0.0,0.0,0.0,True,0.0,108.43504,7.968939,22
3,1985-05-30 18:00:00,9.4815,65.710370,0.0,0.0,0.0,False,0.0,146.52370,26.758923,22
4,1985-05-31 00:00:00,11.9815,61.171284,0.0,0.0,0.0,True,0.0,146.79341,23.664352,22


In [25]:
preprocessing(weather_slice)

,date,temperature_2m,relative_humidity_2m,rain,snowfall,snow_depth,is_day,precipitation,wind_direction_100m,wind_speed_100m,city_id
0,1985-05-30 00:00:00,10.7315,68.325270,0.0,0.0,0.0,True,0.0,259.56256,13.910169,22
1,1985-05-30 06:00:00,18.4315,39.305890,0.0,0.0,0.0,True,0.0,260.53775,4.379589,22
2,1985-05-30 12:00:00,16.8315,55.358337,0.0,0.0,0.0,True,0.0,108.43504,7.968939,22
3,1985-05-30 18:00:00,9.4815,65.710370,0.0,0.0,0.0,False,0.0,146.52370,26.758923,22
4,1985-05-31 00:00:00,11.9815,61.171284,0.0,0.0,0.0,True,0.0,146.79341,23.664352,22


--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Index: 1223175 entries, 0 to 58439
Data columns (total 11 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   date                  1223175 non-null  datetime64[ns]
 1   temperature_2m        1223175 non-null  float64       
 2   relative_humidity_2m  1223175 non-null  float64       
 3   rain                  1223175 non-null  float64       
 4   snowfall              1223175 non-null  float64       
 5   snow_depth            1223175 non-null  float64       
 6   is_day                1223175 non-null  bool          
 7   precipitation         1223175 non-null  float64       
 8   wind_direction_100m   1223175 non-null  float64       
 9   wind_speed_100m       1223175 non-null  float64       
 10  city_id               1223175 non-null  int64         
dtypes: bool(1), datetime64[ns](1), float64(8), int64(1)
memory

,date,temperature_2m,relative_humidity_2m,rain,snowfall,snow_depth,precipitation,wind_direction_100m,wind_speed_100m,city_id
count,1223175,1.223175e+06,1.223175e+06,1.223175e+06,1.223175e+06,1.223175e+06,1.223175e+06,1.223175e+06,1.223175e+06,1.223175e+06
mean,2005-05-05 20:36:10.083270400,2.900022e-01,7.122073e+01,2.112760e-01,5.580686e-02,1.643717e-01,2.759525e-01,2.168799e+02,1.670030e+01,5.469676e+02
min,1985-05-30 00:00:00,-5.332450e+01,6.862968e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.304920e-01,0.000000e+00,2.200000e+01
25%,1995-05-18 06:00:00,-1.107850e+01,6.099963e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.425237e+02,9.885262e+00,1.320000e+02
50%,2005-05-05 18:00:00,1.200000e+00,7.408424e+01,0.000000e+00,0.000000e+00,3.000000e-02,0.000000e+00,2.380000e+02,1.616797e+01,4.080000e+02
75%,2015-04-24 00:00:00,1.280650e+01,8.319903e+01,0.000000e+00,0.000000e+00,2.900000e-01,0.000000e+00,2.965650e+02,2.254531e+01,9.860000e+02
max,2025-06-05 18:00:00,3.722150e+01,1.000000e+02,6.220000e+01,1.281000e+01,1.150000e+00,6.220000e+01,3.600000e+02,7.168973e+01,1.083000e+03
std,NaN,1.495263e+01,1.661604e+01,1.019141e+00,2.274695e-01,2.239776e-01,1.064649e+00,9.188035e+01,8.737548e+00,4.030191e+02


--------------------------------------------------
Явных дубликатов: 0
--------------------------------------------------
Пропусков в колонках:
date                    0
temperature_2m          0
relative_humidity_2m    0
rain                    0
snowfall                0
snow_depth              0
is_day                  0
precipitation           0
wind_direction_100m     0
wind_speed_100m         0
city_id                 0
dtype: int64
--------------------------------------------------
Размер таблицы
(1223175, 11)


In [29]:
total_size_mb = weather_slice.memory_usage(deep=True).sum() / (1024 ** 2)
print(f"Общий размер данных по Иркутской области: {total_size_mb:.4f} Mb")

Общий размер данных по Иркутской области: 103.8194 Mb


In [35]:
# Сохраняем обработанные данные
df.to_csv('weather_slice.csv', index=False, compression="gzip",  encoding='utf-8')
print("\nОбработка данных завершена. Результат сохранен в weather_slice.csv")


Обработка данных завершена. Результат сохранен в weather_slice.csv


In [37]:
csv_size = os.path.getsize('weather_slice.csv') / (1024 ** 2)  # в Мб
print(f"DataFrame: {total_size_mb:.2f} Mb | CSV: {csv_size:.2f} Mb")

DataFrame: 103.82 Mb | CSV: 1257.24 Mb


In [38]:
for col in weather_slice.columns:
    col_size = weather_slice[col].astype(str).str.len().sum() / (1024 ** 2)  # в МБ
    print(f"{col}: {col_size:.2f} MB")

date: 22.16 MB
temperature_2m: 7.57 MB
relative_humidity_2m: 8.94 MB
rain: 3.66 MB
snowfall: 3.89 MB
snow_depth: 4.11 MB
is_day: 5.24 MB
precipitation: 3.69 MB
wind_direction_100m: 9.43 MB
wind_speed_100m: 9.23 MB
city_id: 3.61 MB


In [40]:
weather_slice.to_feather('weather_slice.feather')
csv_size = os.path.getsize('weather_slice.feather') / (1024 ** 2)  # в Мб
print(f"DataFrame: {total_size_mb:.2f} Mb | Feather: {csv_size:.2f} Mb")

DataFrame: 103.82 Mb | Feather: 42.15 Mb
